#### Jupyter Notebook to generate noisy images, segment, and evaluate error.


#### Imports

In [228]:
import numpy as np
from skimage import io
from scipy.misc import imread, imsave
import os
import imageio
import json
from skimage import transform
from skimage.filters import gaussian
import matplotlib.pyplot as plt
from shutil import copy
from AxonDeepSeg.apply_model import axon_segmentation
from AxonDeepSeg.testing.segmentation_scoring import *
%matplotlib inline

#### Function definitions

In [208]:
def add_additive_gaussian_noise(img,mu=0,sigma=10):

    # Generate random gaussian noise with specified mean and sigma values
    noise = np.random.normal(mu, sigma, img.shape)
    
    # Add generated noise to input image
    img_noise = np.add(img,noise)

    # Clip noisy image between 0-255
    img_noise[img_noise < 0] = 0
    img_noise[img_noise > 255] = 255

    return img_noise.astype(np.uint8)

In [209]:
def add_multiplicative_gaussian_noise(img,mu=1,sigma=0.05):

    # Generate random gaussian noise with specified mean and sigma values
    noise = np.random.normal(mu, sigma, img.shape)
    
    # Add generated noise to input image
    img_noise = np.multiply(img,noise)

    # Clip noisy image between 0-255
    img_noise[img_noise < 0] = 0
    img_noise[img_noise > 255] = 255

    return img_noise.astype(np.uint8)

In [210]:
def change_illumination(img,value_percentage=0.2):

    # Add generated noise to input image
    img_noise = img+(value_percentage*255)

    # Clip noisy image between 0-255
    img_noise[img_noise < 0] = 0
    img_noise[img_noise > 255] = 255

    return img_noise.astype(np.uint8)

#### Path to change (put path to image to segment)

In [211]:
path_img = '/Users/Documents/Aldo/axondeepseg/AxonDeepSeg/data_test/image.png'

#### Set paths for AxonDeepSeg segmentation tools

In [221]:
img = imageio.imread(path_img)
path_folder, file_name = os.path.split(path_img)

model_name = 'default_SEM_model_v1'
path_model = os.path.join('..','AxonDeepSeg','models',model_name)
path_configfile = os.path.join(path_model,'config_network.json')
with open(path_configfile, 'r') as fd:
    config_network = json.loads(fd.read())
    
mask = imread(os.path.join(path_folder,'mask.png'),flatten=True)
gt_axon = mask > 200
gt_myelin = np.logical_and(mask >= 50, mask <= 200)

#### 1) Batch for simulation of illumination change:

- For all noise values, generates noisy image, creates sub-folder, saves noisy image, launches AxonDeepSeg segmentation on noisy image, computes and saves prediction accuracy.

In [222]:
values = np.linspace(0,0.7,11)

for value_percentage in values:
    
    # Generate and save noisy image
    img_noise = change_illumination(img,value_percentage)
    file_name = 'image_' + str(int(value_percentage*100)) + '.png'
    directory_name = 'image_' + str(int(value_percentage*100))
    os.makedirs(os.path.join(path_folder,directory_name))
    copy(os.path.join(path_folder,'pixel_size_in_micrometer.txt'), os.path.join(path_folder,directory_name))
    imageio.imwrite(os.path.join(path_folder,directory_name,file_name),img_noise)
    
    # Segment noisy image with AxonDeepSeg
    path_testing = os.path.join(path_folder,directory_name)
    prediction = axon_segmentation(path_testing, file_name, path_model, config_network,verbosity_level=0)
    
    # Compute prediction accuracy
    prediction = imread(os.path.join(path_folder,directory_name,'AxonDeepSeg.png'),flatten=True)
    pred_axon = prediction > 200
    pred_myelin = np.logical_and(prediction >= 50, prediction <= 200)
    dice_axon, dice_myelin = pw_dice(pred_axon, gt_axon),pw_dice(pred_myelin, gt_myelin)
    dict_metrics = {'dice_axon': dice_axon, 'dice_myelin': dice_myelin}
    f = open(os.path.join(path_folder,directory_name,'metrics.txt'), 'w')
    f.write('segmentation metrics: ' + repr(dict_metrics) + '\n')
    f.close()

#### 2) Batch for simulation of gaussian additive noise:

- For all noise values, generates noisy image, creates sub-folder, saves noisy image, launches AxonDeepSeg segmentation on noisy image, computes and saves prediction accuracy.

In [224]:
values = np.linspace(0,30,4)

for sigma_values in values:
    
    # Generate and save noisy image
    img_noise = add_additive_gaussian_noise(img,0,sigma_values)
    file_name = 'image_' + str(int(sigma_values)) + '.png'
    directory_name = 'image_add_' + str(int(sigma_values))
    os.makedirs(os.path.join(path_folder,directory_name))
    copy(os.path.join(path_folder,'pixel_size_in_micrometer.txt'), os.path.join(path_folder,directory_name))
    imageio.imwrite(os.path.join(path_folder,directory_name,file_name),img_noise)
    
    # Segment noisy image with AxonDeepSeg
    path_testing = os.path.join(path_folder,directory_name)
    prediction = axon_segmentation(path_testing, file_name, path_model, config_network,verbosity_level=0)
    
    # Compute prediction accuracy
    prediction = imread(os.path.join(path_folder,directory_name,'AxonDeepSeg.png'),flatten=True)
    pred_axon = prediction > 200
    pred_myelin = np.logical_and(prediction >= 50, prediction <= 200)
    dice_axon, dice_myelin = pw_dice(pred_axon, gt_axon),pw_dice(pred_myelin, gt_myelin)
    dict_metrics = {'dice_axon': dice_axon, 'dice_myelin': dice_myelin}
    f = open(os.path.join(path_folder,directory_name,'metrics.txt'), 'w')
    f.write('segmentation metrics: ' + repr(dict_metrics) + '\n')
    f.close()

#### 3) Batch for simulation of gaussian multiplicative noise:

- For all noise values, generates noisy image, creates sub-folder, saves noisy image, launches AxonDeepSeg segmentation on noisy image, computes and saves prediction accuracy.

In [227]:
values = np.linspace(0,0.5,4)

for sigma_values in values:
    
    # Generate and save noisy image
    img_noise = add_multiplicative_gaussian_noise(img,1,sigma_values)
    file_name = 'image_' + str(int(sigma_values*100)) + '.png'
    directory_name = 'image_mul_' + str(int(sigma_values*100))
    os.makedirs(os.path.join(path_folder,directory_name))
    copy(os.path.join(path_folder,'pixel_size_in_micrometer.txt'), os.path.join(path_folder,directory_name))
    imageio.imwrite(os.path.join(path_folder,directory_name,file_name),img_noise)
    
    # Segment noisy image with AxonDeepSeg
    path_testing = os.path.join(path_folder,directory_name)
    prediction = axon_segmentation(path_testing, file_name, path_model, config_network,verbosity_level=0)
    
    # Compute prediction accuracy
    prediction = imread(os.path.join(path_folder,directory_name,'AxonDeepSeg.png'),flatten=True)
    pred_axon = prediction > 200
    pred_myelin = np.logical_and(prediction >= 50, prediction <= 200)
    dice_axon, dice_myelin = pw_dice(pred_axon, gt_axon),pw_dice(pred_myelin, gt_myelin)
    dict_metrics = {'dice_axon': dice_axon, 'dice_myelin': dice_myelin}
    f = open(os.path.join(path_folder,directory_name,'metrics.txt'), 'w')
    f.write('segmentation metrics: ' + repr(dict_metrics) + '\n')
    f.close()